In [134]:
#autopilot based on minimap path
import cv2 as cv
import numpy as np
from PIL import ImageGrab #screen rec
from IPython.display import clear_output
import keyboard 
import time #used to determine how much to turn, sort of pwm kb input

def thresholding(miniMap):
    #lower and max value of red to create the mask
    lower = np.array([0,239,207])
    upper = np.array([1,240,208])
    #creating mask
    mask = cv.inRange(hsv, lower, upper)
    #applying mask
    res = cv.bitwise_and(miniMap,miniMap, mask = mask)
    
    return mask, res
#----------------------------------------------------------------------------
def getHist(img, minPer = 0.1, region = 1):
    #based on the number of coloured pixel for each column, this function calculates the basePoint which is the "mean" of each column
    if region == 1:
        histVal = np.sum(img, axis = 0)
    else:
        histVal = np.sum(img[img.shape[0]//region:,:], axis = 0)
        
    maxVal = np.max(histVal)
    minVal = minPer * maxVal
    indexArray = np.where(histVal >= minVal)
    basePoint = np.average(indexArray)
    imgHist = np.zeros((img.shape[0], img.shape[1], 3), np.uint8)
    
    for x,intensity in enumerate(histVal):
        cv.line(imgHist,(x, img.shape[0]), (x,img.shape[0]-intensity//255//region) , (255,0,100), 1) #lines to draw the histogram
        cv.circle(imgHist, (int(basePoint), img.shape[0] - 30), 25, (0,80,220), cv.FILLED) #ball placed in basepoint coordinates

    return basePoint, imgHist
#---------------------------------------------------------------------------
def turn(key, difference):
    keyboard.press(key)
    time.sleep(difference/300)
    keyboard.release(key)
#---------------------------------------------------------------------------
while True:
    clear_output()
    miniMap = np.array(ImageGrab.grab(bbox=(2160,1100,2342,1160)))
    miniMap = cv.cvtColor(miniMap, cv.COLOR_BGR2RGB)
    
    mask, res = thresholding(miniMap)[0], thresholding(miniMap)[1]
    
    midPoint, imgHist = getHist(mask, minPer=0.5, region=10)
    
    #print(midPoint)
    
    if midPoint >= 93:
        difference = midPoint - 93
        turn("d", difference)
    elif midPoint <= 91.5:
        difference = 91.5 - midPoint
        turn("a", difference)
    
    cv.imshow('img', miniMap)
    cv.imshow('mask', mask)
    cv.imshow('res', res)
    cv.imshow('imgHist', imgHist)
    
    k = cv.waitKey(5)
    if k == 113:
        break
        
cv.destroyAllWindows()